In [ ]:
#Louvain Alogorithm
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from community import community_louvain
from networkx.algorithms.community.centrality import girvan_newman

# Read the file and build the graph
graphs = {}
with open('facebook/facebook_combined.txt') as f:
    G = nx.Graph()
    for line in f:
        u, v = map(int, line.strip().split())
        G.add_edge(u, v)

# 10 sub-networks
for i in [0, 107, 348, 414, 686, 698, 1684, 1912, 3437, 3980]:
#for i in [0, 107]:
    edges = []
    feat = []
    featnames = []
    
    with open('facebook/{}.edges'.format(i)) as f:
        for line in f:
            u, v = map(int, line.strip().split())
            edges.append((u, v))
    
    with open('facebook/{}.feat'.format(i)) as f:
        for line in f:
            row = list(map(int, line.strip().split()))
            feat.append(row)
        # add featnames
        with open('facebook/{}.featnames'.format(i)) as f:
            for line in f:
                featnames.append(line.strip())
    #Builf sub-graph            
    G_i = nx.Graph()
    G_i.add_edges_from(edges)

    #Create a dictionary of node features
    node_feat_dict = {i+1:feat[i] for i in range(len(feat))}
    # set node attributes
    nx.set_node_attributes(G_i, node_feat_dict, 'feat')
    graphs[i] = G_i

    # Conducting subgraph community partition and calculate the modularity
    partition = community_louvain.best_partition(graphs[i])
    modularity = community_louvain.modularity(partition, graphs[i])
    print(f'Modularity of subnetwork {i} ：', modularity)
    
    # Plotting graph of sub-networks 
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(graphs[i], seed=42)
    cmap = plt.get_cmap('tab20')
    colors = [mcolors.to_rgba(cmap(partition[i])) for i in graphs[i].nodes()]
    nx.draw_networkx_nodes(graphs[i], pos, node_size=50, node_color=colors)
    nx.draw_networkx_edges(graphs[i], pos, alpha=0.5)
    plt.axis('off')
    plt.show()
    '''
    #Print every community in the subnetwork
    for com in set(partition.values()):
        print("Community %d:" % com)
        members = [nodes for nodes in partition.keys() if partition[nodes] == com]
        print(members)
    '''
    # Combined the sub-networks
    G.add_edges_from(edges)

# Partition the whole network and calculate the modularity
pos = nx.spring_layout(G, seed=42)
partition = community_louvain.best_partition(G)
modularity = community_louvain.modularity(partition, G)
print('Modularity of the full network ：', modularity)
plt.figure(figsize=(20, 20))
pos = nx.spring_layout(G, seed=42)

# Plot the complete network graph
for i in [0, 107, 348, 414, 686, 698, 1684, 1912, 3437, 3980]:
#for i in [0, 107]:
    partition = community_louvain.best_partition(graphs[i])
    cmap = plt.get_cmap('tab20')
    colors = [mcolors.to_rgba(cmap(partition[j])) for j in graphs[i].nodes()]
    nx.draw_networkx_nodes(graphs[i], pos, node_size=50, node_color=colors)
    nx.draw_networkx_edges(graphs[i], pos, alpha=0.5)

plt.axis('off')
plt.show()

In [ ]:
#Girvan-Newman alogorithm
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from community import community_louvain
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community.quality import modularity

# Read the file and build the graph
graphs = {}
G = nx.Graph()

with open('facebook/facebook_combined.txt') as f:
    for line in f:
        u, v = map(int, line.strip().split())
        G.add_edge(u, v)
# 10 sub-networks
for i in [0, 107, 348, 414, 686, 698, 1684, 1912, 3437, 3980]:
#for i in [0,107]:
    edges = []
    circles = []
    egofeat = []
    feat = []
    featnames = []
    
    with open('facebook/{}.edges'.format(i)) as f:
        for line in f:
            u, v = map(int, line.strip().split())
            edges.append((u, v))
    
    with open('facebook/{}.feat'.format(i)) as f:
        for line in f:
            row = list(map(int, line.strip().split()))
            feat.append(row)
        # add featnames
        with open('facebook/{}.featnames'.format(i)) as f:
            for line in f:
                featnames.append(line.strip())
    #Builf sub-graph 
    G_i = nx.Graph()
    G_i.add_edges_from(edges)
    graphs[i] = G_i

    #create a dictionary of node features
    node_feat_dict = {i+1:feat[i] for i in range(len(feat))}
    # set node attributes
    nx.set_node_attributes(graphs[i], node_feat_dict, 'feat')

    # Conducting subgraph community partition and calculate the modularity
    comp = girvan_newman(graphs[i])
    comp = sorted(next(comp))
    partition = {}
    for k, c in enumerate(comp):
        for node in c:
            partition[node] = k
    modularity = community_louvain.modularity(partition, graphs[i])
    print(f'Modularity of subnetwork {i} ：', modularity)

    # Plotting graph of sub-networks 
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(G_i, seed=42)
    cmap = plt.get_cmap('tab20')
    colors = [mcolors.to_rgba(cmap(partition[i])) for i in G_i.nodes()]
    nx.draw_networkx_nodes(G_i, pos, node_size=50, node_color=colors)
    nx.draw_networkx_edges(G_i, pos, alpha=0.5)
    plt.axis('off')
    plt.show()

# Combined the subplots
G_all = nx.compose_all(graphs.values())

# Partition the whole network and calculate the modularity
comp = girvan_newman(G_all)
comp = sorted(next(comp))
partition = {}
for k, c in enumerate(comp):
    for node in c:
        partition[node] = k
modularity = community_louvain.modularity(partition, G_all)
print('Modularity of full network：', modularity)

# Plot the complete network graph
plt.figure(figsize=(20, 20))
pos = nx.spring_layout(G_all, seed=42)
cmap = plt.get_cmap('tab20')
colors = [mcolors.to_rgba(cmap(partition[i])) for i in G_all.nodes()]
nx.draw_networkx_nodes(G_all, pos, node_size=50, node_color=colors)
nx.draw_networkx_edges(G_all, pos, alpha=0.5)
plt.axis('off')
plt.show()